In [1]:
import pennylane as qml
from pennylane import numpy as qnp

from  qiskit import IBMQ

## Load your IBM Q Account

An IBM Q account and secret API token are needed to execute quantum circuits on IBM’s hardware and remote simulators. To obtain your IBM Q API token follow these steps:

1. Sign in or create an IBM Quantum account at https://quantum-computing.ibm.com/.
2. Copy the secret IBM Q API token from the welcome page of your profile.
3. Below the token, select `View account Details`, scroll to the `Providers` section and make note of the `hub`, `group`, and `project`. These details are needed to 
4. Uncomment the first chunk of the following code and edit the `token`, `hub`, `group`, and `project` fields with the correct information. The example shows how Xanadu's IBM Q startup account is loaded.

You only need to enable your account once. Afterwards, it'll be stored locally. and you can simply run `provider = IBMQ.load_account()`.

In [2]:
# NOTE : uncomment the next two lines if setting up your account for the first time.

# token = "XYZ"   # secret IBM Q API token from above
# provider = IBMQ.enable_account(
#     token=token, hub="ibm-q-startup", group="xanadu", project="reservations"
# )

# NOTE : use the next line if you have previously enabled your account using the above lines

provider = IBMQ.load_account()

## Setting up a Simple Quantum Circuit

The first step is to construct a PennyLane device that will run our circuit.
To start, we'll use IBM's remote `ibmq_qasm_simulator` device.
This device simulates IBM's quantum computers, but without the queuing wait times and significant noise on quantum hardware.
A successful run on IBM's simulator means that quantum hardware can be also be run with success.

In [3]:
sim_dev_kwargs = {
    "name": "qiskit.ibmq",
    "wires": [0,1],
    "shots": 1000,
    "backend": "ibmq_qasm_simulator",
    "provider": provider,
}
sim_dev = qml.device(**sim_dev_kwargs)

Now, we will construct a simple circuit to run on IBM's simulator device.
We'll create a Bell state with local qubit rotations on it.
The `diff_method="parameter-shift"` argument specifies that when the gradient is evaluated for the quantum circuit PennyLane will use the parameter-shift rule to evaluate analytical gradients on quantum hardware.

In [4]:
@qml.qnode(sim_dev, diff_method="parameter-shift")
def bell_state_circ(settings):
    qml.Hadamard(wires=sim_dev.wires[0])
    qml.CNOT(wires=sim_dev.wires[0:2])

    qml.RY(settings[0], wires=sim_dev.wires[0])
    qml.RY(settings[1], wires=sim_dev.wires[1])

    return qml.probs(wires=sim_dev.wires[0:2])

Next, we will run the circuit for two different settings.
The next cell will take a few seconds to run because of the latency of remote execution.
If everything is setup correctly, a probability distribution close to `[0.5, 0, 0, 0.5]` will be returned for the first case, and in the second case, `[0.25, 0.25, 0.25, 0.25]`.
You can then `view all` jobs in your IBM Q account and verify that the circuit was run.
Selecting the job will show more info such as the circuit executed by IBM, the output data, and a timeline of execution including validation, wait time, and execution time.

In [5]:
bell_state_probs1 = bell_state_circ([0, 0])
print("bell_state_probs1 = ", bell_state_probs1)

bell_state_probs2 = bell_state_circ([qnp.pi/2, 0])
print("bell_state_probs2 = ", bell_state_probs2)

bell_state_probs1 =  [0.532 0.    0.    0.468]


bell_state_probs2 =  [0.235 0.267 0.232 0.266]


Finally, we can process the data to evaluate a qubit covariance matrix that we can use to infer network topology.
Note that the covariance matrices of each case are quite distinct although both cases are maximally entangled states.
This shows measurement dependence of the qubit covariance matrix.

In [6]:
cov_mat1 = qml.math.cov_matrix(
    bell_state_probs1,
    obs = [qml.PauliZ(0), qml.PauliZ(1)],
)

print("covariance matrix 1 : ")
print(cov_mat1)

cov_mat2 = qml.math.cov_matrix(
    bell_state_probs2,
    obs = [qml.PauliZ(0), qml.PauliZ(1)],
)

print("covariance matrix 2 : ")
print(cov_mat2)

covariance matrix 1 : 
[[0.995904 0.995904]
 [0.995904 0.995904]]
covariance matrix 2 : 
[[0.999984 0.002264]
 [0.002264 0.995644]]


## Optimizing the Covariance Matrix for Topology Inference

In this section, we will use the Bell state example to show how the parameter-shift rule can be used to optimize quantum circuits run on quatum hardware.
We first construct a `cost` function that when minimized, will maximize the square of the distance between the covariance matrix and the origin, *i.e.*, we boost the off-diagonals of the matrix to yield visibility into correlative structure of the network.

In [7]:
def cost(*settings):
    probs = bell_state_circ(settings)
    cov_mat = qml.math.cov_matrix(probs, obs = [qml.PauliZ(0), qml.PauliZ(1)])

    return -qnp.trace(cov_mat.T @ cov_mat)

Now, we will set up a basic gradient descent routine and run it on IBM's quantum computer simulator.
Note that this may take some time because we'll evaluate the cost in each step alongside the gradient. I count 2 executions in each optimization step.
First, we set some variables.

In [11]:
init_settings = 2*qnp.pi * (0.5 - qnp.random.rand(2, requires_grad=True))
opt = qml.GradientDescentOptimizer(stepsize=0.1)

settings_list = []
cost_vals = []

init_settings

tensor([2.95437356, 1.244625  ], requires_grad=True)

Now we run the optimization!

In [12]:
settings = init_settings
for i in range(6):
    settings, cost_val = opt.step_and_cost(cost, *settings)

    settings_list += [settings]
    cost_vals += [cost_val]

    print("\ngradient descent iteration : ", i)
    print("cost val : ", cost_val)
    print("settings : ", settings)


gradient descent iteration :  0
cost val :  -2.0407152143999996
settings :  [tensor(3.01310563, requires_grad=True), tensor(1.18705473, requires_grad=True)]



gradient descent iteration :  1
cost val :  -2.0446030062240004
settings :  [tensor(3.07800963, requires_grad=True), tensor(1.12287572, requires_grad=True)]



gradient descent iteration :  2
cost val :  -2.266550323776
settings :  [tensor(3.21606604, requires_grad=True), tensor(0.98525808, requires_grad=True)]



gradient descent iteration :  3
cost val :  -2.6035478209439997
settings :  [tensor(3.38836584, requires_grad=True), tensor(0.80484469, requires_grad=True)]



gradient descent iteration :  4
cost val :  -3.4381687056000003
settings :  [tensor(3.56343261, requires_grad=True), tensor(0.63600337, requires_grad=True)]



gradient descent iteration :  5
cost val :  -3.8942021736959997
settings :  [tensor(3.64519603, requires_grad=True), tensor(0.54996084, requires_grad=True)]


Overall, what you should see in the optimization is two things.
1. The cost value approaches `-4` which is is maximum value for the given circuit (each element of the covariance matrix is $\pm$ 1).
2. The optimal settings are arbitrary, but at optimality, `settings[0] == settings[1]`.

For validation, we also return the optimal covariance matrix, which is evaluate from the probabilities obtaained by executing `bell_state_circ(opt_settings)`  on the remote IBM simulator.

In [13]:
min_id = qnp.argmin(cost_vals)
min_cost = cost_vals[min_id]

opt_settings = settings_list[min_id]

print("minimal cost : ", min_cost)
print("optimal settings : ", opt_settings)
print("optimal covariance matrix : ")
print(qml.math.cov_matrix(
    bell_state_circ(opt_settings),
    obs = [qml.PauliZ(0), qml.PauliZ(1)],
))

minimal cost :  -3.8942021736959997
optimal settings :  [tensor(3.64519603, requires_grad=True), tensor(0.54996084, requires_grad=True)]
optimal covariance matrix : 


[[ 0.999964 -0.997976]
 [-0.997976  0.999984]]
